In [1]:
# include shiny and plotly packages
library(shiny)
library(plotly)

# install and use RQuantlib package
# if (!require("drat")) install.packages("drat")
# drat::addRepo("ghrr")
# install.packages("RQuantLib", type="binary")
library(RQuantLib)

# sample usage of RQuantlib package
# AmericanOption('call', 100, 100,5, 5, 1, 2)

# library(Rcpp)
# evalCpp("1 + 1", showOutput= TRUE)

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'plotly'

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout

Warning message:
"package 'RQuantLib' was built under R version 3.6.3"

In [2]:
# Generalized Black Scholes model for pricing vanilla European options
# Compute values of call and put options as well as the Greeks

#' Find Black Scholes option prices and Greeks/Advanved Greeks values.
#' 
#' @param s current stock price.
#' @param X strike price.
#' @param r risk free interest rate percetage.
#' @param Sigma standard deviation of stock.
#' @param t time untill expiry in years.
#' @param div_yield dividend yield percentage.
#'
#' @return option call price.
#' @return option put price.
#' @return greek/advanced greek values.
#' @return risk neutral probabilities.
#'
#' @examples
#' black_scholes(100, 100, 5, 20, 1, 2)
#'
black_scholes <- function(s, X, r, Sigma, t, div_yield)
{
    # the growth rate of the asset price
    g = r - div_yield 
    
    # the division below can be 0/0=NaN if s=X and either t=0 or g=Sigma=0
    # we set d1 to 0 in this case since numerator is of higher order in t and Sigma
    d1 <- (log(s / X) + (g + Sigma * Sigma / 2) * t) / (Sigma * sqrt(t))
    d1 <- ifelse(is.nan(d1), 0, d1)
    d2 <- d1 - Sigma * sqrt(t)
    
    # dnorm: density function of the normal distribution
    # pnorm: cumulative density function of the normal distribution
    Nd1 <- pnorm(d1)
    Nd2 <- pnorm(d2)
    Nminusd1 <- pnorm(-d1)
    Nminusd2 <- pnorm(-d2)
    
    # Black Scholes call and put price
    call <- exp(-r * t) * (s * exp(g * t) * Nd1 - X * Nd2)
    put <- exp(-r * t) * (-s * exp(g * t) * Nminusd1 + X * Nminusd2)
    
    # BASIC GREEKS
    
    callDelta <- exp(-div_yield * t) * Nd1
    putDelta <- -exp(-div_yield * t) * Nminusd1
    
    # if t is 0, the below is 0/0 = NaN but dnorm(d1) term goes to 0 faster
    # so we set the result to 0 in this case
    a <- -s * dnorm(d1) * Sigma * exp(-div_yield*t) / (2 * sqrt(t))
    a <- ifelse(is.nan(a), 0,  a)
    
    b = r * X * exp(-r * t) * Nd2
    c = div_yield * s * Nd1* exp(-div_yield*t)
    callTheta <- a - b + c
    
    b = r * X * exp(-r * t) * Nminusd2
    c = div_yield * s * Nminusd1 * exp(-div_yield*t)
    putTheta <- a + b - c
    
    # if t is 0, the below is 0/0 = NaN but dnorm(d1) term goes to 0 faster
    # so we set the result to 0 in this case
    Gamma <- dnorm(d1) * exp(-div_yield*t)/ (s * Sigma * sqrt(t))
    Gamma <- ifelse(is.nan(Gamma), 0,  Gamma)
    Vega <- s * sqrt(t) * dnorm(d1) * exp(-div_yield*t)
    callRho <- X * t * exp(-r * t) * Nd2
    putRho <- -X * t * exp(-r * t) * Nminusd2

    # ADVANCED GREEKS
    
    Vanna<- -exp(-div_yield*t)*dnorm(d1)*d2/Sigma 
    Vanna <- ifelse(is.nan(Vanna), 0,  Vanna)
    callCharm<- div_yield*exp(-div_yield*t)*Nd1-exp(-div_yield*t)*dnorm(d1)*
                                ((2*(r-div_yield)*t-d2*Sigma*sqrt(t))/(2*t*Sigma*sqrt(t)))
    callCharm <- ifelse(is.nan(callCharm), 0,  callCharm)
    putCharm<- -div_yield*exp(-div_yield*t)*Nminusd1-exp(-div_yield*t)*dnorm(d1)*
                                ((2*(r-div_yield)*t-d2*Sigma*sqrt(t))/(2*t*Sigma*sqrt(t)))
    putCharm <- ifelse(is.nan(putCharm), 0,  putCharm)
    Speed<- -exp(-div_yield*t)*dnorm(d1)*(d1/(Sigma*sqrt(t)) +1)/(s*s*Sigma*sqrt(t))
    Speed <- ifelse(is.nan(Speed), 0,  Speed)
    Color<- -exp(-div_yield*t)*dnorm(d1)*
                        (2*div_yield*t +1 + d1*(2*(r-div_yield)*t-d2*Sigma*sqrt(t))/(Sigma*sqrt(t)))/(2*s*t*Sigma*sqrt(t)) 
    Color <- ifelse(is.nan(Color), 0,  Color)
    Zomma<- exp(-div_yield*t)*dnorm(d1)*(d1*d2 -1)/(s*Sigma*Sigma*sqrt(t))
    Zomma <- ifelse(is.nan(Zomma), 0,  Zomma)
    Vomma<- s*exp(-div_yield*t)*dnorm(d1)*sqrt(t)*d1*d2/Sigma
    Vomma <- ifelse(is.nan(Vomma), 0,  Vomma)

    callProb <- Nd2
    putProb <- Nminusd2
    
    Greeks <- list(callDelta=callDelta, putDelta=putDelta, callTheta=callTheta, putTheta=putTheta, 
                   Gamma=Gamma, Vega=Vega, callRho=callRho, putRho=putRho,Vanna=Vanna, callCharm=callCharm,
                   putCharm=putCharm,Speed=Speed,Color=Color,Zomma=Zomma,Vomma=Vomma)
    extra <- list(d1=d1, d2=d2, Nd1=Nd1, Nd2=Nd2, Nminusd1=Nminusd1, Nminusd2=Nminusd2, 
                  callProb=callProb, putProb=putProb)
    
    list(call=call, put=put, Greeks=Greeks, extra=extra)
}

# Generalized Black Scholes model implied volatility

#' Find implied volatility of option.
#' 
#' @param s current stock price.
#' @param X strike price.
#' @param r risk free interest rate percetage.
#' @param price option price.
#' @param t time untill expiry in years.
#' @param div_yield dividend yield percentage.
#' @param optType option type - call or put.
#' @param tolerence for inbuilt uniroot function.
#' @param max_iter maximum number of iterations for inbuilt uniroot function.
#' @param convergence for inbuilt uniroot function.
#'
#' @return implied volatility of the option
#'
#' @examples
#' implied_volatility(100, 100, 5, 10, 1, 2, Call)
#'
implied_volatility <- function(s, X, r, price, t, div_yield, optType, tolerance=1e-6, max_iter=100, convergence=1e-8)
{
    # discount the exercise price to eliminate r
    X = X * exp(-r * t)
    
    # adjust the spot price to eliminate div_yield
    s = s * exp(-div_yield * t)
    
    # use put call parity to convert put option into call option
    if (optType=="Put") price = price + (s - X)
    
    SminusX = s - X
    SplusX = s + X
    
    if (price < SminusX || price < 0 || price > s)
    {
        warning("Implied volatility is undefined because price is outside theoretical bounds")
        return(NA)
    }
    
    # if price equals intrinsic value, volatility is zero
    if (price == SminusX || price == 0) 
        return(0)
    
    # if x is 0, option price must equal stock price
    if (X == 0 && price != s) 
    {
        warning("Implied volatility is undefined because price is outside theoretical bounds")
        return(NA)
    }
    
    # since price exceeds intrinsic value, 0 is a lower bound for the volatility
    # we now seek an upper bound by doubling 100% volatility until the price is exceeded
    
    upper <- 100e-2
    while (black_scholes(s, X, r, upper, t, div_yield)$call < price && upper < .Machine$integer.max) 
        upper <-  upper * 2
    if (upper > .Machine$integer.max) 
        upper <-  Inf
    
    f <- function(sigma)
    {
        temp <- black_scholes(s, X, r, sigma, t, div_yield)
        temp$call - price
    }
    
    res2 <- uniroot(f, c(0, upper), tol=convergence)
    if (abs(res2$f.root > tolerance))
    {
        warning("Error finding implied volatility")
        return (NA)                         
    }
    
    return (res2$root)
}


In [3]:
# Tab 1 : Black Scholes values
# Tab 2 : BLack Scholes Greek
# Tab 3 : Advanced Greek
# Tab 4 : Combination of Options

# S, X, r, div_yield, sigma, t have their usual meanings (except specified otherwise)

# number behind stands for the tab number to which the input/output object corresponds

ui <- fluidPage(
        titlePanel("Generalized Black Scholes App: Value and Greeks for Call and Put Options"),
        sidebarLayout
        (
            sidebarPanel
            (
                conditionalPanel
                (
                    condition = "input.conditionedPanels == 1",
                    numericInput('S1', 'Spot', 100),
                    numericInput('X1', 'Strike', 100),
                    numericInput('r1', 'Rf (%)', 5),
                    numericInput("div_yield1", "Dividend Yield (%)", 2),
                    numericInput("sigma1", "Volatility (%)", 20),
                    sliderInput("t1", "Time to Maturity (Years)", value = 1, 
                                min = 0, max = 5, step = 0.1), 
                    uiOutput("radioButtonSelect1"),
                    radioButtons('Implied', NULL, 
                                 list('Find Option Price' = FALSE,
                                      'Find Implied Volatility' = TRUE), 
                                      selected = FALSE, inline = TRUE)
                ),
                
                conditionalPanel
                (
                    condition = "input.conditionedPanels == 2",
                    numericInput('S2', 'Spot', 100),
                    numericInput('X2', 'Strike', 100),
                    numericInput('r2', 'Rf (%)', 5),
                    numericInput("div_yield2", "Dividend Yield (%)", 2),
                    numericInput("sigma2", "Volatility (%)", 20),
                    sliderInput("t2", "Time to Maturity (Years)", value = 1, 
                                min = 0, max = 5, step = 0.1), 
                    radioButtons
                    (
                        "greek", "Select Greek to Plot",
                         c("Delta" = "Delta","Gamma" = "Gamma","Rho" = "Rho", 
                           "Theta" = "Theta","Vega" = "Vega")
                    )
                ),
                
                conditionalPanel
                (
                    condition = "input.conditionedPanels == 3",
                    numericInput('S3', 'Spot', 100),
                    numericInput('X3', 'Strike', 100),
                    numericInput('r3', 'Rf (%)', 5),
                    numericInput("div_yield3", "Dividend Yield (%)", 2),
                    numericInput("sigma3", "Volatility (%)", 20),
                    sliderInput("t3", "Time to Maturity (Years)", value = 1, 
                                min = 0, max = 5, step = 0.1), 
                    radioButtons
                    (
                        "advGreek", "Select Advanced Greek to Plot",
                         c("Vanna" = "Vanna", "Charm" = "Charm", "Speed" = "Speed",
                           "Color" = "Color","Zomma" = "Zomma","Vomma"="Vomma")
                    )
                ),
                
                conditionalPanel
                (
                    condition = "input.conditionedPanels == 4",
                    selectInput("numOpt", 
                                "Select no. of Options for combination", 
                                c(1,2,3,4)),
                    uiOutput(outputId = "combInput"),
                    numericInput('S4', 'Spot', 100),
                    numericInput('r4', 'Rf (%)', 5),
                    numericInput("div_yield4", "Dividend Yield (%)", 2),
                    numericInput("sigma4", "Volatility (%)", 20),
                    sliderInput("t4", "Time to Maturity (Years)", value = 1, 
                                min = 0, max = 5, step = 0.1), 
                    checkboxGroupInput("optional_display4a", 
                                       "Select plots to display:",
                                        c("Value","Profit"), c("Value")),
                    checkboxGroupInput("optional_display4b", 
                                       "Select additional information about Greeks to display:",
                                       c("GreekValues",
                                         "GreekPlots",
                                         "AdvancedGreekValues",
                                         "AdvancedGreekPlots")),
                    uiOutput("radioButtonSelect4a"),
                    uiOutput("radioButtonSelect4b")
                 ),
                
                actionButton("reset", "RESET")
            ),

            mainPanel
            (
                tabsetPanel
                (
                    type = "tabs",
                    tabPanel
                    (
                        "Black Scholes Values", value = 1,
                         h3(textOutput("title1a")),
                         tableOutput("table1"), hr(),
                         h3(textOutput("title1b")),
                         fluidRow(splitLayout(cellWidths = c("50%", "50%"), 
                                              plotlyOutput("callPlot"), 
                                              plotlyOutput("putPlot")))
                    ),
                    
                    tabPanel
                    (
                        "Black Scholes Greeks", value = 2,
                         h3('Option Greeks for selected Inputs'), 
                         tableOutput("table2"), hr(),
                         h3('Plot of selected Greek'), 
                         plotlyOutput("plot2"), br(),  
                         h5(paste("Note: Gamma and Vega are same for both Call",
                                 "and Put options (from Put Call parity)"))
                    ),
                    
                    tabPanel
                    (
                        "Advanced Greeks", value = 3,
                         h3('Advanced option Greeks for selected Inputs'), 
                         tableOutput("table3"), hr(),
                         h3('Plot of selected Greek'), 
                         plotlyOutput("plot3"), br(),  
                         h5(paste("Note: All advanced greeks except Charm are",
                                 "same for both Call and Put options",
                                 "(because of Put-Call parity)"))
                    ),
                    
                    tabPanel
                    (
                        "Combination of Options", value = 4,
                         h3('Payoff for selected combination of Inputs'), 
                         plotlyOutput("plot4"), br(), 
                         fluidRow(splitLayout(cellWidths = c("50%", "50%"), 
                                              h3(textOutput("title4a")), 
                                              h3(textOutput("title4c")))), 
                         fluidRow(splitLayout(cellWidths = c("50%", "50%"), 
                                              tableOutput("table4a"), 
                                              tableOutput("table4b"))),
                         fluidRow(splitLayout(cellWidths = c("50%", "50%"), 
                                              h3(textOutput("title4b")), 
                                              h3(textOutput("title4d")))), 
                         fluidRow(splitLayout(cellWidths = c("50%", "50%"), 
                                              plotlyOutput("plot4a"), 
                                              plotlyOutput("plot4b"))), 
                         h5(paste("Note: We assume that all options have the",
                                 "same maturity, current spot price,",
                                 "interest rate, dividend yield and",
                                 "volatility. Only the strike price differs."))
                    ),
                            
                    id = "conditionedPanels"
                )
            )
        ),
        
        title ="Generalized Black Scholes"
    )

In [4]:
# Tab 1 : Black Scholes values
# Tab 2 : BLack Scholes Greek
# Tab 3 : Advanced Greek
# Tab 4 : Combination of Options

# number behind stands for the tab number to which the input/output object corresponds

server <- function(input, output, session) 
{ 
    # RESET button for all tabs
    observe(
        {
            req(input$reset)
            
            updateRadioButtons(session, 'Implied',
                               NULL, list('Find Option Price' = FALSE,
                                          'Find Implied Volatility' = TRUE),
                               selected = FALSE, inline = TRUE)
            
            updateNumericInput(session, "S1", value = 100)
            updateNumericInput(session, "S2", value = 100)
            updateNumericInput(session, "S3", value = 100)
            updateNumericInput(session, "S4", value = 100)
            
            updateNumericInput(session, "X1", value = 100)
            updateNumericInput(session, "X2", value = 100)
            updateNumericInput(session, "X3", value = 100)
            
            updateNumericInput(session, "r1", value = 5)
            updateNumericInput(session, "r2", value = 5)
            updateNumericInput(session, "r3", value = 5)
            updateNumericInput(session, "r4", value = 5)
            
            updateNumericInput(session, "div_yield1", value = 2)
            updateNumericInput(session, "div_yield2", value = 2)
            updateNumericInput(session, "div_yield3", value = 2)
            updateNumericInput(session, "div_yield4", value = 2)
            
            updateNumericInput(session, "t1", value = 1)
            updateNumericInput(session, "t2", value = 1)
            updateNumericInput(session, "t3", value = 1)
            updateNumericInput(session, "t4", value = 1)
            
            updateNumericInput(session, "sigma1",
                               label = "Volatility (%)", value = 20)
            updateNumericInput(session, "sigma2",
                               label = "Volatility (%)", value = 20)
            updateNumericInput(session, "sigma3",
                               label = "Volatility (%)", value = 20)
            updateNumericInput(session, "sigma4",
                               label = "Volatility (%)", value = 20)
            
            updateSelectInput(session, "numOpt",
                              "Select no. of Options for combination",
                              c(1, 2, 3, 4))
            
            updateSelectInput(session, "type1",
                              "Opt Type 1", c("Call" = "Call", "Put" = "Put"))
            updateNumericInput(session, "num1", "Opt Qty 1", value = 1)
            updateNumericInput(session, "pri1",
                               "Opt Strike 1", value = 100)
            
            
            updateRadioButtons(session,"greek", "Select Greek to Plot",
                               c("Delta" = "Delta", "Gamma" = "Gamma",
                                 "Rho" = "Rho", "Theta" = "Theta",
                                 "Vega" = "Vega"))
            
            updateRadioButtons(session,"advGreek",
                               "Select Advanced Greek to Plot",
                               c("Vanna" = "Vanna", "Charm" = "Charm",
                                 "Speed" = "Speed", "Color" = "Color",
                                 "Zomma" = "Zomma","Vomma"="Vomma"))
            
            updateCheckboxGroupInput(session, "optional_display1",
                                     "Select plots to display:",
                                     c("Value","Profit"), c("Value"))
            updateCheckboxGroupInput(session, "optional_display4a",
                                     "Select plots to display:",
                                     c("Value", "Profit"), c("Value"))
            updateCheckboxGroupInput(session, "optional_display4b",
                                     "Select additional information about Greeks to display:",
                                     c("GreekValues", "GreekPlots",
                                       "AdvancedGreekValues",
                                       "AdvancedGreekPlots"))
        }
    ) 
    
    
    
    # TAB 1 : BLACK SCHOLES VALUES
    
    spots1 <- reactive(seq(0.2 * input$S1, 1.8 * input$S1, 0.01 * input$S1))
    results1 <- reactive(black_scholes(spots1(), input$X1,input$r1/100, 
                                       input$sigma1/100, input$t1, 
                                       input$div_yield1/100))
    
    # Change what sigma1 stands for based on whether to display price or volatility
    observe(
        {
          if(input$Implied) {
              updateNumericInput(session, "sigma1",
                                 label = "Option Price", value = 10)
          } else {
              updateNumericInput(session, "sigma1",
                                 label = "Volatility (%)", value = 20)
          }      
        }
    )
    
    # Provide extra inputs based on whether to display price or volatility
    output$radioButtonSelect1 <- renderUI(
        { 
            if(input$Implied) {
                selectInput("optType", "Option Type",
                            c("Call" = "Call", "Put" = "Put"))
            } else {
                checkboxGroupInput("optional_display1", 
                                   "Select plots to display:",
                                    c("Value","Profit"), c("Value"))
            }
        }
    )
    
    # Display title of table based on whether to display price or volatility
    output$title1a <- renderText(
        { 
            if(input$Implied) {
                "Implied Volatity for selected Inputs"
            } else {
                "Option Values for selected Inputs"
            }
        }
    )
    
    # Display title of plot if price is to be displayed and not volatility
    output$title1b <- renderText(
        { 
            if(input$Implied) {
                ""
            } else{
                "Plot of Call and Put Options"
            }
        }
    )
    
    # Display values in table based on whether to display price or volatility
    output$table1 <- renderTable({
      if(input$Implied) {
          req(input$optType)
          Sigma <- implied_volatility(input$S1, input$X1,
                                      input$r1/100, input$sigma1,
                                      input$t1, input$div_yield1/100,
                                      input$optType)
          dfV <- data.frame(Option = format(c(Sigma * 100), nsmall = 4,
                                            digits = 4),
                            row.names = c('Implied Volatility (%)'))
          col.name <- input$optType
          names(dfV)[names(dfV) == "Option"] <- col.name
      } else {
          sig <- input$sigma1 / 100
          res <- black_scholes(input$S1, input$X1, input$r1/100,
                               sig, input$t1, input$div_yield1/100)
          dfV <- data.frame(Call = format(c(res$call, res$extra$callProb),
                                          nsmall = 4, digits = 4), 
                            Put = format(c(res$put, res$extra$putProb),
                                         nsmall = 4, digits = 4), 
                            row.names = c('Value', 'Risk Neutral Probability'))
      }
     t(dfV)
    }, rownames = TRUE)
    
    # Plot call value, payoff, profit for a range of inputs
    output$callPlot <- renderPlotly(
        {
            if(input$Implied) {
            } else {
                if ("Profit" %in% input$optional_display1) {
                    valueC <- results1()$call
                    payoffC <- pmax(spots1() - input$X1, 0)
                    profitC <- payoffC - res$call
                    dfC <- data.frame(spots1(), valueC, payoffC, profitC)
                    plot_ly(dfC, x = ~spots1(),
                            y = ~payoffC,
                            name = 'Call Payoff',
                            type = 'scatter', mode='lines') %>%
                        add_trace(y = ~valueC, name = 'Call Value',
                                  mode='lines') %>%
                        add_trace(y = ~profitC, name = 'Call Profit',
                                  mode='lines') %>%
                        layout(xaxis = list(title = "Underlying"),
                               yaxis = list(title = "Call Option Payoff and Profit"),
                               legend = list(x = 100, y = 0.5))
                } else {
                    if ("Value" %in% input$optional_display1) {
                        valueC <- results1()$call
                        payoffC <- pmax(spots1() - input$X1, 0)
                        dfC <- data.frame(spots1(), valueC, payoffC)
                        plot_ly(dfC, x = ~spots1(), y = ~payoffC,
                                name = 'Call Payoff',
                                type = 'scatter', mode='lines') %>%
                            add_trace(y = ~valueC, name = 'Call Value',
                                      mode='lines') %>%
                            layout(xaxis = list(title = "Underlying"),
                                   yaxis = list(title = "Call Option Payoff and Value"),
                                   legend = list(x = 100, y = 0.5))
                    } else {
                        payoffC <- pmax(spots1() - input$X1, 0)
                        dfC <- data.frame(spots1(), payoffC)
                        plot_ly(dfC, x = ~spots1(), y = ~payoffC,
                                name = 'Call Payoff', type = 'scatter',
                                mode='lines') %>%
                            layout(xaxis = list(title = "Underlying"),
                                   yaxis = list(title = "Call Option Payoff"),
                                   legend = list(x = 100, y = 0.5))
                    } 
                }
             }
        }
    )

    # Plot put value, payoff, profit for a range of inputs
    output$putPlot <- renderPlotly(
        {
            if(input$Implied) {
            } else {
                if("Profit" %in% input$optional_display1)
                {
                    res <- black_scholes(input$S1, input$X1, input$r1/100, 
                                         input$sigma1/100, input$t1,
                                         input$div_yield1/100)
                    valueP <- results1()$put
                    payoffP <- pmax(input$X1 - spots1(), 0)
                    profitP <- payoffP - res$put
                    dfP <- data.frame(spots1(), payoffP, profitP)
                    plot_ly(dfP, x = ~spots1(), y = ~payoffP, name = 'Put Payoff',
                            type = 'scatter', mode='lines') %>%
                        add_trace(y = ~valueP, name = 'Put Value',
                                  mode='lines') %>%
                        add_trace(y = ~profitP, name = 'Put Profit',
                                  mode='lines') %>%
                    layout(xaxis = list(title = "Underlying"),
                           yaxis = list(
                               title = "Put Option Payoff and Profit"),
                           legend = list(x = 100, y = 0.5))  
                } else {
                    if("Value" %in% input$optional_display1) {
                        valueP <- results1()$put
                        payoffP <- pmax(input$X1 - spots1(), 0)
                        dfP <- data.frame(spots1(), valueP, payoffP)
                        plot_ly(dfP, x = ~spots1(), y = ~payoffP,
                                name = 'Put Payoff', type = 'scatter',
                                mode='lines') %>%
                            add_trace(y = ~valueP, name = 'Put Value',
                                      mode='lines') %>%
                        layout(xaxis = list(title = "Underlying"),
                               yaxis = list(
                                   title = "Put Option Payoff and Value"),
                               legend = list(x = 100, y = 0.5))
                    } else {
                        payoffP <- pmax(input$X1 - spots1(), 0)
                        dfP <- data.frame(spots1(), payoffP)
                        plot_ly(dfP, x = ~spots1(), y = ~payoffP,
                                name = 'Put Payoff', type = 'scatter',
                                mode='lines') %>%
                        layout(xaxis = list(title = "Underlying"),
                               yaxis = list(title = "Put Option Payoff"),
                               legend = list(x = 100, y = 0.5))
                    } 
                }
            }
        }
    )
    
    
    # Tab 2 : Black Scholes Greeks
    
    spots2 <- reactive(seq(0.2 * input$S2, 1.8 * input$S2, 0.01 * input$S2))
    results2 <- reactive(black_scholes(spots2(), input$X2, input$r2/100,
                                       input$sigma2/100, input$t2,
                                       input$div_yield2/100))
    
    # Display table with values of greeks for given inputs
    output$table2 <- renderTable(
        {
            res <- black_scholes(input$S2, input$X2, input$r2/100,
                         input$sigma2/100, input$t2, input$div_yield2/100)
            dfG <- data.frame(Call = format(c(res$Greeks$callDelta,
                                              res$Greeks$callTheta,
                                              res$Greeks$Gamma,
                                              res$Greeks$Vega,
                                              res$Greeks$callRho),
                                            nsmall = 4, digits = 4),
                              Put = format(c(res$Greeks$putDelta,
                                             res$Greeks$putTheta,
                                             res$Greeks$Gamma,
                                             res$Greeks$Vega,
                                             res$Greeks$putRho),
                                           nsmall = 4, digits = 4),
                              row.names = c('Delta', 'Theta',
                                            'Gamma', 'Vega', 'Rho'))
            t(dfG)
        }, 
        rownames = TRUE
    )
    
    callData2 <- reactive(
        {
            switch(
                input$greek,
                Delta = results2()$Greeks$callDelta,
                Gamma = results2()$Greeks$Gamma,
                Rho = results2()$Greeks$callRho,                        
                Theta = results2()$Greeks$callTheta,
                Vega = results2()$Greeks$Vega,
                results2()$Greeks$callDelta
            )
        }
    )
    
    putData2 <- reactive(
        {
            switch(
                input$greek,
                Delta = results2()$Greeks$putDelta,
                Gamma = results2()$Greeks$Gamma,
                Rho = results2()$Greeks$putRho,                        
                Theta = results2()$Greeks$putTheta,
                Vega = results2()$Greeks$Vega,
                results2()$Greeks$putDelta
            )
        }
    )

    # Plot values of greeks for range of input
    output$plot2 <- renderPlotly(
        {
            greek <- input$greek
            dfG <- data.frame(spots2(), callData2(), putData2())
            plot_ly(dfG, x = ~spots2(), y = ~callData2(),
                    name = paste("Call", greek), type = 'scatter',
                    mode='lines') %>%
                add_trace(y = ~putData2(), name = paste("Put", greek),
                          mode='lines') %>%
            layout(xaxis = list(title = "Underlying"),
                   yaxis = list(title = paste("Call and Put Option", greek)),
                   legend = list(x = 100, y = 0.5))
        }
    )
    
    
    # Tab 3 : Advanced Greeks
    
    spots3 <- reactive(seq(0.2 * input$S3, 1.8 * input$S3, 0.01 * input$S3))
    results3 <- reactive(black_scholes(spots3(), input$X3, input$r3/100,
                                       input$sigma3/100, input$t3,
                                       input$div_yield3/100))
    
    # Display table with values of advanced greeks for given inputs
    output$table3 <- renderTable(
        {
            res <- black_scholes(input$S3, input$X3,
                         input$r3/100, input$sigma3/100,
                         input$t3, input$div_yield3/100)
            dfG <- data.frame(Call = format(c(res$Greeks$Vanna,
                                              res$Greeks$callCharm,
                                              res$Greeks$Speed,
                                              res$Greeks$Color,
                                              res$Greeks$Zomma,
                                              res$Greeks$Vomma),
                                            nsmall = 4, digits = 4),
                              Put = format(c(res$Greeks$Vanna,
                                             res$Greeks$putCharm,
                                             res$Greeks$Speed,
                                             res$Greeks$Color,
                                             res$Greeks$Zomma,
                                             res$Greeks$Vomma),
                                           nsmall = 4, digits = 4),
                              row.names = c('Vanna','Charm','Speed',
                                            'Color','Zomma','Vomma'))
            t(dfG)
        }, 
        rownames = TRUE
    )
    
    callData3 <- reactive(
        {
            switch(
                input$advGreek,
                Vanna = results3()$Greeks$Vanna,
                callCharm = results3()$Greeks$callCharm,
                Speed = results3()$Greeks$Speed,
                Color = results3()$Greeks$Color,
                Zomma = results3()$Greeks$Zomma,
                Vomma = results3()$Greeks$Vomma,
                results3()$Greeks$callDelta
            )
        }
    )
    
    putData3 <- reactive(
        {
            switch(
                input$advGreek,
                Vanna = results3()$Greeks$Vanna,
                putCharm = results3()$Greeks$putCharm,
                Speed = results3()$Greeks$Speed,
                Color = results3()$Greeks$Color,
                Zomma = results3()$Greeks$Zomma,
                Vomma = results3()$Greeks$Vomma,
                results3()$Greeks$putDelta
            )
        }
    )

    # Plot vales of advanved greeks for range of input
    output$plot3 <- renderPlotly(
        {
            advGreek <- input$advGreek
            dfG2 <- data.frame(spots3(), callData3(), putData3())
            plot_ly(dfG2, x = ~spots3(), y = ~callData3(),
                    name = paste("Call", advGreek),
                    type = 'scatter', mode='lines') %>%
                add_trace(y = ~putData3(), name = paste("Put", advGreek),
                          mode='lines') %>%
            layout(xaxis = list(title = "Underlying"),
                   yaxis = list(title = paste("Call and Put Option", advGreek)),
                   legend = list(x = 100, y = 0.5))
        }
    )
    
    
    # Tab 4 : Combnation of options
    
    spots4 <- reactive(seq(0.2 * input$S4, 1.8 * input$S4, 0.01 * input$S4))
    
    # Display inputs based on  number on options in combination
    output$combInput <- renderUI({
        numinputs <- lapply(seq(length.out = req(as.numeric(input$numOpt))),
                            function(i) {
                                fluidRow(
                                    column(4,
                                           selectInput(inputId =
                                                           paste0("type", i),
                                                       label =
                                                           paste0("Opt Type ", i),
                                                       c("Call" = "Call",
                                                         "Put" = "Put"))
                                           ), 
                                    column(4,
                                           numericInput(inputId =
                                                            paste0("num", i),
                                                        label =
                                                            paste0("Opt Qty", i),
                                                        value = 1)
                                           ),
                                    column(4,
                                           numericInput(inputId =
                                                            paste0("pri", i),
                                                        label =
                                                            paste0("Opt Strike ", i),
                                                        value = 100)
                                           )
                                )
                            })
    })
    
    # Plot value, payoff, profit of combination for range of input
    output$plot4 <- renderPlotly(
        {
            payoff <- 0
            value <- 0
            val <- 0
            c_n <- c(input$num1, input$num2, input$num3, input$num4)
            c_ty <- c(input$type1, input$type2, input$type3, input$type4)
            c_pri <- c(input$pri1, input$pri2, input$pri3, input$pri4)
            
            for(i in 1:input$numOpt) {
                req(c_n[i])
                req(c_ty[i])
                req(c_pri[i])
                results4 <- reactive(black_scholes(spots4(), c_pri[i],
                          input$r4 / 100, input$sigma4/100,
                          input$t4, input$div_yield4/100))
                res <- reactive(black_scholes(input$S4, c_pri[i],
                                       input$r4/100, input$sigma4/100,
                                       input$t4, input$div_yield4/100))
                    
                if(c_ty[i] == "Call") {
                    payoff <- payoff + c_n[i] * pmax(spots4() - c_pri[i], 0)
                    value <- value + c_n[i] * results4()$call
                    val <- val + c_n[i] * res()$call
                } else {
                    payoff <- payoff + c_n[i] * pmax(c_pri[i] - spots4(), 0)
                    value <- value + c_n[i] * results4()$put
                    val <- val + c_n[i] * res()$put
                }
            } 
            
            profit <- payoff - val
            
            if("Profit" %in% input$optional_display4a) {
                 dfC <- data.frame(spots4(), value, payoff)
                 plot_ly(dfC, x = ~spots4(), y = ~payoff,
                         name = 'Combination Payoff', type = 'scatter',
                         mode='lines') %>%
                     add_trace(y = ~value,
                               name = 'Combination Value',
                               mode='lines') %>%
                     add_trace(y = ~profit,
                               name = 'Combination Profit',
                               mode='lines') %>%
                    layout(xaxis = list(title = "Underlying"),
                           yaxis = list(
                               title = "Combination Payoff, Value and Profit"),
                           legend = list(x = 100, y = 0.5))                   
            } else {
                if("Value" %in% input$optional_display4a) {
                    dfC <- data.frame(spots4(), value, payoff)
                    plot_ly(dfC, x = ~spots4(), y = ~payoff,
                            name = 'Combination Payoff', type = 'scatter',
                            mode='lines') %>%
                        add_trace(y = ~value, name = 'Combination Value',
                                  mode='lines') %>%
                    layout(xaxis = list(title = "Underlying"),
                           yaxis = list(
                               title = "Combination Payoff and Value"),
                           legend = list(x = 100, y = 0.5))
                } else {
                    dfC <- data.frame(spots4(), value, payoff)
                    plot_ly(dfC, x = ~spots4(), y = ~payoff,
                            name = 'Combination Payoff', type = 'scatter',
                            mode='lines') %>%
                    layout(xaxis = list(title = "Underlying"),
                           yaxis = list(title = "Combination Payoff"),
                           legend = list(x = 100, y = 0.5))
                }
                
            }
            
            
        }
    )
    
    # Display title of greeks value table
    output$title4a <- renderText(
        { 
            if("GreekValues" %in% input$optional_display4b) {
                "Greek Values for combination of selected Inputs"
            }
        }
    )
    
    # Display title of greeks plot
    output$title4b <- renderText(
        { 
            if("GreekPlots" %in% input$optional_display4b) {
                "Plot of selected Greek"
            }
        }
    )
    
    # Display title of advanced greeks values table
    output$title4c <- renderText(
        { 
            if("AdvancedGreekValues" %in% input$optional_display4b) {
                "Advanced Greek Values for combination of selected Inputs"
            }
        }
    )
    
    # Display title of advanced greeks values plot
    output$title4d <- renderText(
        { 
            if("AdvancedGreekPlots" %in% input$optional_display4b) {
                "Plot of selected Advanced Greek"
            }
        }
    )
    
    # Display inputs to select which greek plot is to be displayed
    output$radioButtonSelect4a <- renderUI(
        { 
            if("GreekPlots" %in% input$optional_display4b) {
                radioButtons("greek4", "Select Greek to Plot",
                             c("Delta" = "Delta", "Gamma" = "Gamma",
                               "Rho" = "Rho", "Theta" = "Theta",
                               "Vega" = "Vega"))
            }
         }
    )
    
    # Display inputs to select which advanced greek plot is to be displayed
    output$radioButtonSelect4b <- renderUI(
        { 
            if("AdvancedGreekPlots" %in% input$optional_display4b) {
                radioButtons("advGreek4", "Select Advanced Greek to Plot",
                             c("Vanna" = "Vanna", "Charm" = "Charm",
                               "Speed" = "Speed", "Color" = "Color",
                               "Zomma" = "Zomma", "Vomma"="Vomma"))
            }
        }
    )
    
    # Display greek values table
    output$table4a <- renderTable(
        {
            if("GreekValues" %in% input$optional_display4b) {
                c_n <- c(input$num1, input$num2, input$num3, input$num4)
                c_ty <- c(input$type1, input$type2, input$type3, input$type4)
                c_pri <- c(input$pri1, input$pri2, input$pri3, input$pri4)
                DeltaComb <- 0
                ThetaComb <- 0
                GammaComb <- 0
                VegaComb <- 0
                RhoComb <- 0
                for(i in 1:input$numOpt) {
                    req(c_n[i])
                    req(c_pri[i])
                    req(c_ty[i])
                    results4 <- reactive(black_scholes(spots4(), c_pri[i],
                          input$r4/100, input$sigma4/100,
                          input$t4, input$div_yield4/100))
                    res <- reactive(black_scholes(input$S4, c_pri[i],
                                                  input$r4/100, input$sigma4/100,
                                                  input$t4, input$div_yield4/100))
                    
                    if(c_ty[i] == "Call") {
                        DeltaComb <-  DeltaComb +
                            c_n[i] * res()$Greeks$callDelta
                        ThetaComb <- ThetaComb +
                            c_n[i] * res()$Greeks$callTheta
                        RhoComb <- RhoComb +
                            c_n[i] * res()$Greeks$callRho
                    } else {
                        DeltaComb <-  DeltaComb +
                            c_n[i] * res()$Greeks$putDelta
                        ThetaComb <- ThetaComb +
                            c_n[i] * res()$Greeks$putTheta
                        RhoComb <- RhoComb +
                            c_n[i] * res()$Greeks$putRho
                    }
                    GammaComb <- GammaComb +
                        c_n[i] * res()$Greeks$Gamma
                    VegaComb <- VegaComb +
                        c_n[i] * res()$Greeks$Vega
                } 

                dfG <- data.frame(Combo = format(c(DeltaComb,
                                                   ThetaComb,
                                                   GammaComb,
                                                   VegaComb,
                                                   RhoComb),
                                                 nsmall = 4, digits = 4),
                                  row.names = c('Delta', 'Theta',
                                                'Gamma', 'Vega', 'Rho'))
                t(dfG) 
            }
            
        }, 
        rownames = TRUE
    )

    # Display advanced greek values table
    output$table4b <- renderTable(
        {
            if("AdvancedGreekValues" %in% input$optional_display4b) {
                c_n <- c(input$num1, input$num2, input$num3, input$num4)
                c_ty <- c(input$type1, input$type2, input$type3, input$type4)
                c_pri <- c(input$pri1, input$pri2, input$pri3, input$pri4)
                VannaComb <- 0
                CharmComb <- 0
                SpeedComb <- 0
                ColorComb <- 0
                ZommaComb <- 0
                VommaComb <- 0
                putCharmComb <- 0
                for(i in 1:input$numOpt) {
                    req(c_n[i])
                    req(c_pri[i])
                    req(c_ty[i])
                    results4 <- reactive(black_scholes(spots4(), c_pri[i],
                                          input$r4 / 100, input$sigma4 / 100,
                                          input$t4, input$div_yield4 / 100))
                    res <- reactive(black_scholes(input$S4, c_pri[i],
                                                   input$r4 / 100, input$sigma4 / 100,
                                                   input$t4, input$div_yield4 / 100))
                    
                    if(c_ty[i] == "Call") {
                        CharmComb <- CharmComb +
                            c_n[i] * res()$Greeks$callCharm
                    } else {
                        CharmComb <- CharmComb + c_n[i] * res()$Greeks$putCharm
                    }
                    VannaComb <- VannaComb +
                        c_n[i] * res()$Greeks$Vanna
                    SpeedComb <- SpeedComb +
                        c_n[i] * res()$Greeks$Speed
                    ColorComb <- ColorComb +
                        c_n[i] * res()$Greeks$Color
                    ZommaComb <- ZommaComb +
                        c_n[i] * res()$Greeks$Zomma
                    VommaComb <- VommaComb +
                        c_n[i] * res()$Greeks$Vomma
                    
                }

                dfG <- data.frame(Combo = format(c(VannaComb,
                                                   CharmComb,
                                                   SpeedComb,
                                                   ColorComb,
                                                   ZommaComb,
                                                   VommaComb),
                                                 nsmall = 4, digits = 4),
                                  row.names = c('Vanna','Charm',
                                                'Speed','Color',
                                                'Zomma','Vomma'))
                t(dfG)
            }
        }, 
        rownames = TRUE
    )
    
    # Plot greeks of combination for range of input
    output$plot4a <- renderPlotly(
        {
            if("GreekPlots" %in% input$optional_display4b) {
                req(input$greek4)
                greek4 <- input$greek4                
                c_n <- c(input$num1, input$num2, input$num3, input$num4)
                c_ty <- c(input$type1, input$type2, input$type3, input$type4)
                c_pri <- c(input$pri1, input$pri2, input$pri3, input$pri4)
                for(i in 1:input$numOpt) {
                    req(c_n[i])
                    req(c_pri[i])
                    req(c_ty[i])
                    results4 <- reactive(black_scholes(spots4(), c_pri[i],
                                          input$r4 / 100, input$sigma4 / 100,
                                          input$t4, input$div_yield4 / 100))
                    
                    if(c_ty[i] == "Call"){
                        temp <- c_n[i] * switch(
                                         input$greek4,
                                         Delta = results4()$Greeks$callDelta,
                                         Gamma = results4()$Greeks$Gamma,
                                         Rho = results4()$Greeks$callRho,
                                         Theta = results4()$Greeks$callTheta,
                                         Vega = results4()$Greeks$Vega,
                                         results4()$Greeks$callDelta
                                     )
                        comb <- integer(length(spots4()))
                        comb <- comb + temp
                    } else {
                        temp <- c_n[i] * switch(
                                         input$greek4,
                                         Delta = results4()$Greeks$putDelta,
                                         Gamma = results4()$Greeks$Gamma,
                                         Rho = results4()$Greeks$putRho,
                                         Theta = results4()$Greeks$putTheta,
                                         Vega = results4()$Greeks$Vega,
                                         results4()$Greeks$putDelta
                                     )
                        comb <- integer(length(spots4()))
                        comb <- comb + temp
                    }
                    
                } 
                        
                dfG <- data.frame(spots4(), comb)
                plot_ly(dfG, x = ~spots4(), y = comb,
                        name = paste("Combo", greek4),
                        type = 'scatter', mode='lines') %>%
                layout(xaxis = list(title = "Underlying"),
                       yaxis = list(title = paste("Combinations of Option",
                                                  greek4)),
                       legend = list(x = 100, y = 0.5))
            }
        }
    )
    
    # Plot advanced greeks of combination for range of input
    output$plot4b <- renderPlotly(
        {
            if("AdvancedGreekPlots" %in% input$optional_display4b) {
                req(input$advGreek4)
                advGreek4 <- input$advGreek4
                c_n <- c(input$num1, input$num2, input$num3, input$num4)
                c_pri <- c(input$pri1, input$pri2, input$pri3, input$pri4)
                c_ty <- c(input$type1, input$type2, input$type3, input$type4)
                for(i in 1:input$numOpt) {
                    req(c_n[i])
                    req(c_pri[i])
                    req(c_ty[i])
                    results4 <- reactive(black_scholes(spots4(), c_pri[i],
                          input$r4 / 100, input$sigma4 / 100,
                          input$t4, input$div_yield4 / 100))
                    
                    if(c_ty[i] == "Call"){
                        temp <- c_n[i] * switch(
                                         input$advGreek4,
                                         Vanna = results4()$Greeks$Vanna,
                                         Charm = results4()$Greeks$callCharm,
                                         Speed = results4()$Greeks$Speed,
                                         Color = results4()$Greeks$Color,
                                         Zomma = results4()$Greeks$Zomma,
                                         Vomma = results4()$Greeks$Vomma,
                                         results4()$Greeks$callDelta
                                     )
                        comb <- integer(length(spots4()))
                        comb <- comb + temp
                    } else {
                        temp <- c_n[i] * switch(
                                         input$advGreek4,
                                         Vanna = results4()$Greeks$Vanna,
                                         Charm = results4()$Greeks$putCharm,
                                         Speed = results4()$Greeks$Speed,
                                         Color = results4()$Greeks$Color,
                                         Zomma = results4()$Greeks$Zomma,
                                         Vomma = results4()$Greeks$Vomma,
                                         results4()$Greeks$callDelta
                                     )
                        comb <- integer(length(spots4()))
                        comb <- comb + temp
                    }
                    
                } 
                
                
                dfG <- data.frame(spots4(), comb)
                plot_ly(dfG, x = ~spots4(), y = comb,
                        name = paste("Combo", advGreek4),
                        type = 'scatter', mode='lines') %>%
                layout(xaxis = list(title = "Underlying"),
                       yaxis = list(title = paste("Combination of Option",
                                                  advGreek4)),
                       legend = list(x = 100, y = 0.5))
            }
        }
    )
       
}


In [ ]:
shinyApp(ui=ui,server=server)


Listening on http://127.0.0.1:6127
Warning message:
"Error in promises::is.promising: object 'res' not found"